# 参考CSDN博客： mmdetection 使用自定义的coco格式数据集进行训练及测试

博客地址: <a>https://blog.csdn.net/gaoyi135/article/details/90613895</a>

## 1. 数据集准备

参考`labelme2coco.py`，将`facedata`数据集转化为`coco`格式

In [7]:

import os
import json
import numpy as np
import glob
import shutil
from sklearn.model_selection import train_test_split
np.random.seed(41)

from PIL import Image

import labelme.utils as lu
def originImageLoader(image_path):
    with open(image_path, encoding='utf-8') as json_file:
        labelmeJson = json.load(json_file)
        
        image = lu.img_b64_to_arr(labelmeJson['imageData'])
        image = image.astype(np.uint8)

        return image

#0为背景
classname_to_id = {"opencomedo": 1, "closedcomedo": 2, "papule": 3, "nudule": 4, "scar": 5, "undefined": 6}

class Lableme2CoCo:

    def __init__(self):
        self.images = []
        self.annotations = []
        self.categories = []
        self.img_id = 0
        self.ann_id = 0

    def save_coco_json(self, instance, save_path):
        json.dump(instance, open(save_path, 'w', encoding='utf-8'), ensure_ascii=False, indent=1)  # indent=2 更加美观显示

    # 由json文件构建COCO
    def to_coco(self, json_path_list):
        self._init_categories()
        for json_path in json_path_list:
            obj = self.read_jsonfile(json_path)
            self.images.append(self._image(obj, json_path))
            shapes = obj['shapes']
            for shape in shapes:
                annotation = self._annotation(shape)
                self.annotations.append(annotation)
                self.ann_id += 1
            self.img_id += 1
        instance = {}
        instance['info'] = 'spytensor created'
        instance['license'] = ['license']
        instance['images'] = self.images
        instance['annotations'] = self.annotations
        instance['categories'] = self.categories
        return instance

    # 构建类别
    def _init_categories(self):
        for k, v in classname_to_id.items():
            category = {}
            category['id'] = v
            category['name'] = k
            self.categories.append(category)

    # 构建COCO的image字段
    def _image(self, obj, path):
        image = {}
        from labelme import utils
        img_x = utils.img_b64_to_arr(obj['imageData'])
        h, w = img_x.shape[:-1]
        image['height'] = h
        image['width'] = w
        image['id'] = self.img_id
        image['file_name'] = os.path.basename(path).replace(".json", ".jpg")
        return image

    # 构建COCO的annotation字段
    def _annotation(self, shape):
        label = shape['label']
        points = shape['points']
        annotation = {}
        annotation['id'] = self.ann_id
        annotation['image_id'] = self.img_id
        if label == 'closecomedo':
            annotation['category_id'] = 2
        elif label == 'comedo w':
            annotation['category_id'] = 2
        elif label == 'comedo b':
            annotation['category_id'] = 1
        elif label == 'pigment':
            annotation['category_id'] = 3
        elif label == 'undifined':
            annotation['category_id'] = 6
        elif label == 'nodule':
            annotation['category_id'] = 4
        elif label == 'scar hyper':
            annotation['category_id'] = 5
        else :
            annotation['category_id'] = int(classname_to_id[label])
        annotation['segmentation'] = [np.asarray(points).flatten().tolist()]
        annotation['bbox'] = self._get_box(points)
        annotation['iscrowd'] = 0
        annotation['area'] = 1.0
        return annotation

    # 读取json文件，返回一个json对象
    def read_jsonfile(self, path):
        with open(path, "r", encoding='utf-8') as f:
            return json.load(f)

    # COCO的格式： [x1,y1,w,h] 对应COCO的bbox格式
    def _get_box(self, points):
        min_x = min_y = np.inf
        max_x = max_y = 0
        for x, y in points:
            min_x = min(min_x, x)
            min_y = min(min_y, y)
            max_x = max(max_x, x)
            max_y = max(max_y, y)
        return [min_x, min_y, max_x - min_x, max_y - min_y]


if __name__ == '__main__':
    labelme_path = "/home/wangjy/research/maskrcnn/facedata/"
    saved_coco_path = "../data/facedata/"
    # 创建文件
    if not os.path.exists("%scoco/annotations/"%saved_coco_path):
        os.makedirs("%scoco/annotations/"%saved_coco_path)
    if not os.path.exists("%scoco/images/train2017/"%saved_coco_path):
        os.makedirs("%scoco/images/train2017"%saved_coco_path)
    if not os.path.exists("%scoco/images/val2017/"%saved_coco_path):
        os.makedirs("%scoco/images/val2017"%saved_coco_path)
    # 获取images目录下所有的joson文件列表
    json_list_path = glob.glob(labelme_path + "/*.json")
    # 数据划分,这里没有区分val2017和tran2017目录，所有图片都放在images目录下
    train_path, val_path = train_test_split(json_list_path, test_size=0.12)
    print("train_n:", len(train_path), 'val_n:', len(val_path))

    # 把训练集转化为COCO的json格式
    l2c_train = Lableme2CoCo()
    train_instance = l2c_train.to_coco(train_path)
    l2c_train.save_coco_json(train_instance, '%scoco/annotations/instances_train2017.json'%saved_coco_path)
    for file in train_path:
        # shutil.copy(file.replace("json","jpg"),"%scoco/images/train2017/"%saved_coco_path)
        im = Image.fromarray(originImageLoader(file))
        file_name = file.split("/", 6)[6]
        file_name = file_name.replace("json","jpg")
        im.save("%scoco/images/train2017/"%saved_coco_path + file_name)
    for file in val_path:
        # shutil.copy(file.replace("json","jpg"),"%scoco/images/val2017/"%saved_coco_path)
        im = Image.fromarray(originImageLoader(file))
        file_name = file.split("/", 6)[6]
        file_name = file_name.replace("json","jpg")
        im.save("%scoco/images/val2017/"%saved_coco_path + file_name)

    # 把验证集转化为COCO的json格式
    l2c_val = Lableme2CoCo()
    val_instance = l2c_val.to_coco(val_path)
    l2c_val.save_coco_json(val_instance, '%scoco/annotations/instances_val2017.json'%saved_coco_path)

train_n: 124 val_n: 17


## 2. Training前修改相关文件


1、定义数据种类，需要修改的地方在mmdetection/mmdet/datasets/coco.py。把CLASSES的那个tuple改为自己数据集对应的种类tuple即可。例如：

```python
CLASSES = ('Glass_Insulator', 'Composite_Insulator', 'Clamp', 'Drainage_Plate')
```

2、接着在mmdetection/mmdet/core/evaluation/class_names.py修改coco_classes数据集类别，这个关系到后面test的时候结果图中显示的类别名称。例如：
```python
def coco_classes():
    return [
        'Glass_Insulator', 'Composite_Insulator', 'Clamp', 'Drainage_Plate'
    ]
```

3、修改configs/faster_rcnn_r50_fpn_1x.py中的model字典中的num_classes、data字典中的img_scale和optimizer中的lr(学习率)。例如：
```python
num_classes=5,#类别数+1
img_scale=(640,478), #输入图像尺寸的最大边与最小边（train、val、test这三处都要修改）
optimizer = dict(type='SGD', lr=0.0025, momentum=0.9, weight_decay=0.0001) #当gpu数量为8时,lr=0.02；当gpu数量为4时,lr=0.01；我只要一个gpu，所以设置lr=0.0025
```
4、在mmdetection的目录下新建work_dirs文件夹

> 3中实际修改的是`mask_rcnn_r50_fpn_1x_coco.py`